<a href="https://colab.research.google.com/github/luisfranc123/Tutorials_Statistics_Numerical_Analysis/blob/main/Numerical_Methods/Chapter19_Root_Finding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**19. ROOT FINDING**
---
**Textbook**: Python Programming and Numerical Methods

####**19.1 Root Finding Problem Statement**

The **root** or **zero** of a function, $f(x)$, is an $x_r$ such that $f(x_r)=0$. For functions such as $f(x) = x^2 - 9$, the roots are clearly 3 and -3. However, for other functions such as $f(x) = cos(x) - x$, determining an **analytic** or exact solution for the roots can be difficult. For these cases, it is useful to generate numerical approximations of the roots of *f* and understand their limitations.

**Try it**: Use the `fsolve` function from `Scipy` to compute the root of $f(x) = cos(x) - x$ near -2. Verify that the solution is a root (or close enough).

In [ ]:
import numpy as np
from scipy import optimize

f = lambda x: np.cos(x) - x
r = optimize.fsolve(f, -2)
print(f"r = {r}")

# Verify the solution is a root
result = f(r)
print(f"result = {result}")

r = [0.73908513]
result = [0.]


**Try it!**: The function $f(x) =\frac{1}{x}$ has no root. Use the `fsolve` function to try to compute the rot of it. Turn on the `full_output` to see what is going on.

In [ ]:
import numpy as np
from scipy import optimize

f = lambda x: 1/x
r, infodict, ier , mesg = optimize.fsolve(f, -2, full_output = True)
print(f"r = {r}")

result = f(r)
print(f"Result = {result}")

print(mesg)

r = [-3.52047359e+83]
Result = [-2.84052692e-84]
The number of calls to function has reached maxfev = 400.


####**19.2 Tolerance**

In engineering and science, **error** is a deviation from an expected or computed value. **Tolerance** is the level of error that is acceptable for an engineering application. We say that a computer program has **converged** to a solution whe it has found a solution with an error smaller than the tolerance. When computing roots numerically, or conducting any other kind of numerical analysis, it is important to establish both a metric for error and and a tolerance that is suitable for a given engineering/science application.





####**19.3 Bisection Method**

The **Intermediate Value Theorem** says that if $f(x)$ is a continuous function between $a$ and $b$, and
$sign(f (a))\neq sign(f (b))$, then there must be a $c$, such that $a<c<b$ and $f(c)= 0$.

The **bisection method** uses the intermediate value theorem iteratively to ﬁnd roots. Let $f(x)$ be
a continuous function, and $a$ and $b$ be real scalar values such that $a<b$. Assume, without loss of
generality, that $f(a)>0$ and $f(b)<0$. Then, by the intermediate value theorem, there must be a root
in the open interval $(a, b)$.Now let $m = \frac{b+a}{2}$ be the midpoint between and $a$ and $b$.If $f(m)= 0$ or is close enough, then $m$ is a root. If $f(m)>0$, then $m$ is an improvement on the left bound, a, and it is guaranteed that there is a root in the open interval $(m, b)$. If $f(m)<0$, then $m$ is an improvement on the right bound, $b$, it is guaranteed that there is a root in the open interval $(a, m)$.
The process of updating $a$ and $b$ can be repeated until the error is acceptably low.

<img src = "https://pythonnumericalmethods.studentorg.berkeley.edu/_images/19.03.02-Bisection-method.png" width = "400" height = "350">




**Try it!**: Program a function `my_bisection(f, a, b, tol)` that approximates a root `r` of `f`, bounded by `a` and `b` to within $|f(\frac{a+b}{2})| <$ `tol`.

In [ ]:
import numpy as np

def my_bisection(f, a, b, tol):
  """
   Approximates a root r, of f bounded
   by a and b to within tolerance
   |f(m)| < tol with m being the midpoint
   between a and b. Recursive implementation
  """

  # check if a and b bound a root
  if np.sign(f(a)) == np.sign(f(b)):
    raise Exception(
        "The scalars a and b do not bound a root")

  # get midpoint
  m = (a + b)/2

  if np.abs(f(m)) < tol:
    # stopping condition. Reports m as root
    return m
  elif np.sign(f(a)) == np.sign(f(m)):
    # case where m is an improvement on a.
    # Make recursive call with a = m
    return my_bisection(f, m, b, tol)
  elif np.sign(f(b)) == np.sign(f(m)):
    # case where m is an improvement on b.
    # Make recursive call with b = m
    return my_bisection(f, a, m, tol)

**Try it!**: The $\sqrt{2}$ can be computed as the root of the function $f(x) = x^2 - 2$. Starting at $a = 0$ and $b = 2$,use `my_bisection` to approximate the $\sqrt{2}$ to a tolerance of $|f(x) < 0.1|$ and $|f(x) < 0.01|$. verify that the results are close to a root by plugging the root back into the function.

In [ ]:
f = lambda x: x**2 - 2

r1 = my_bisection(f, 0, 2, 0.1)
print(f"r1 = {r1:.5f}")
r01 = my_bisection(f, 0, 2, 0.01)
print(f"r01 = {r01:.5f}")

print(f"f(r1) = {f(r1):.5f}")
print(f"f(r01) = {f(r01):.5f}")

r1 = 1.43750
r01 = 1.41406
f(r1) = 0.06641
f(r01) = -0.00043


**Try it!**: See what happens when we use $a = 2$ and $b = 4$ for the above function.

In [ ]:
my_bisection(f, 2, 4, 0.1)

Exception: The scalars a and b do not bound a root

####**19.4 Newton-Raphson Method**

Let $f(x)$ be a smooth function, and $x_r$ be an unknown root of $f(x)$. Assume that $x_0$ is a guess for $x_r$. Unless $x_0$ is very lucky guess, $f(x_0)$ will not be a root. Given this scenario, we want to find an $x_1$ that is an improvement on $x_0$ (i.e., closer to $x_r$ than $x_0$). If we assume that $x_0$ is "close enough" to $x_r$, then we can improve upon it by taking the linear approximation of $f(x)$ around $x_0$, which is a line, and finding the intersection of this line with the $x$-axis. Written out, the linear approximation of $f(x)$ around $x_0$ is $f(x) \approx f(x_0)+f^{'}(x_0)(x-x_0)$. Using this approximation, we find $x_1$ sucha that $f(x_1) = 0$. Plugging these values into the linear apprximation results in the following equation:

$$0 = f(x_0) + f^{'}(x_0)(x_1-x_0)$$

which when solved for $x_1$ yields

$$x_1 = x_0 - \frac{f(x_0)}{f^{'}(x_0)}.$$

An ilustration of how this linear approximation improves an initial guess is shown below.

![Newton-Rapshon](https://qph.cf2.quoracdn.net/main-qimg-b3ff8a5f14c4a4e7525533518dbcd7f0)

Written generally, a **Newton step** computes an improved guess, $x_i$, usig a previous guess, $x_{i-1}$, and is given by the equation

$$x_i = x_{i-1} - \frac{g(x_{i-1})}{g^{'}(x_{i-1})}.$$

The **Newton-Raphson Method** of finding roots iterates Newton steps from $x_0$ until the error is less than the tolerance.

**Try it!**: Again, the $\sqrt{2}$ is the root of the functio $f(x) = x^2-2$. Using $x_0= 1.4$ as a starting point, use the previous equation to estimate $\sqrt{2}$. Compare this approximation with the value computed by Python's `sqrt` function.

$$x = 1.4 - \frac{1.4^2-2}{2(1.4)}= 1.4142857...$$





In [ ]:
import numpy as np

f = lambda x: x**2 - 2
f_prime = lambda x: 2*x
newton_raphson = 1.4 - (f(1.4))/(f_prime(1.4))

print(f"Newton-Raphson = {newton_raphson}")
print(f"sqrt(2) = {np.sqrt(2)}")

Newton-Raphson = 1.4142857142857144
sqrt(2) = 1.4142135623730951


**Try it**: Write a function `my_newton(f, dx, x0, tol)` where the output is an estimate of the root of `f`, `f` is a function object $f(x)$, `df` is a function object $f^{'}(x)$, `x0` is an initial guess, and `tol` is the error tolerance. The error measurement should be $|f(x)|$.

In [ ]:
def my_newton_raphson(f, dx, x0, tol):
  """
  Output is an estimate of the root of f
  using the Newton-Raphsod Method
  recursive implementation
  """

  if abs(f(x0)) < tol:
    return x0
  else:
    return my_newton_raphson(f, dx, x0 - f(x0)/dx(x0), tol)

**Try it!**: Use the previous function to compute $\sqrt{2}$ to within a tolerance of $1exp(-6)$ starting at `x0 = 1.5`.

In [ ]:
import numpy as np

f = lambda x: x**2 - 2
f_prime = lambda x: 2*x
estimmate = my_newton_raphson(f, f_prime, 1.5, 1e-6)
print(f"Estimate = {estimmate}")
print(f"sqrt(2) = {np.sqrt(2)}")

Estimate = 1.4142135623746899
sqrt(2) = 1.4142135623730951


If $x_0$ is close to $x_r$, then it can be proven that, in general, the Newton–Raphson method converges
to $x_r$ much faster than the bisection method; however, since $x_r$ is initially unknown, there is no way to know if the initial guess is close enough to the root to obtain this behavior unless some special information about the function is known *a priori* (e.g., the function has a root close to $x = 0$). In addition to this initialization problem, the Newton–Raphson method has other serious limitations. For example, if the derivative at a guess is close to zero, then the Newton step will be very large and probably lead far away from the root. Also, depending on the behavior of the function derivative between $x_0$ and $x_r$, the Newton–Raphson method may converge to a different root than $x_r$ which may not be useful for the engineering application being considered.

**Try it!**: Compute a single Newton step to get an improved approximation of the root of the function $f(x) = x^3 + 3x^2 - 2x - 5$ and initial guess $x_0 = 0.29$.


In [ ]:
x0 = 0.29

f = lambda x: x**3 + 3*x**2 - 2*x - 5
f_prime = lambda x: 3*x**2 + 6*x - 2
x1 = x0 - (f(x0))/(f_prime(x0))
print(f"x1 = {x1}")

x1 = -688.4516883116648


Note that $f^{'}(x_0) = -0.0077$ (close to zero), and the error at $x_1$ is approximately 324880000.

####**19.4 Root finding in Python**

Unsurprisingly, Python has root-finding functions. The function we will use to find roots is `f_solve` from `scipy.optimize`.

The `f_solve` function takes in many arguments (study the documentation for additional information), but the most important two are: (1) the function that we want to find the root and (2) the initial guess.

**Try it!**: Compute the root of the function $f(x) = x^3 -100x^2 + 100$ using `f_solve`.

In [ ]:
from scipy.optimize import fsolve

f = lambda x: x**3 - 100*x**2 - x + 100

fsolve(f, [2, 80])

array([  1., 100.])

####**19.6 Summary and Problems**

**Summary**
1. Roots are an important property of functions.
2. The bisection method is a way of ﬁnding roots based on divide-and-conquer. Although stable, it
might converge slowly compared to the Newton–Raphson method.
3. The Newton–Raphson method is a different way of ﬁnding roots based on an approximation of the
function. Although the Newton–Raphson method converges quickly and stops near to the actual root, it can be unstable.

**Problems**


1. Write a function `my_nth_root(x, n, tol)` where `x` and `tol` are strictly positive scalars, and `n` is an integer strictly greather than 1. The output argument, `r`, should be an approximation $r = \sqrt[n]{x}$, the $N$th root of `x`. This approximation should be computed by using the Newton-Raphson method to find the root of the function $f(y)=y^N - x$. The error metric should be $|f(y)|$.   

In [ ]:
def my_nth_root(x, n, tol):
  # Define the function and
  # its associated derivative

  # raise an exception
  if x < 0 and n < 0:
    raise Exception(
       "The scalars x and n must be greater than 0")


  f = lambda y: y**n - x
  f_prime = lambda y: n*y**(n-1)

  if abs(f(x)) < tol:
    return x
  else:
    return my_nth_root(x - (f(x))/(f_prime(x)), n, tol)

In [ ]:
import numpy as np
x = 2
n = 2.5
tol = 1e-6
estimate = my_nth_root(2, 2, 1e-6)
true_value = x**(1/n)
print(f"Estimate = {estimate}")
print(f"True Value = {true_value}")
print(f"Error = {(abs(estimate - true_value)/true_value)}")

Estimate = 1.0000009536743164
True Value = 1.3195079107728942
Error = 0.2421409939948207


In [ ]:
f = lambda y: y**n - x
x = 2
n = 1
f(x)

0

**2.** Write a function `my_fixed_point(f, g, tol, max_iter)` where `f` and `g` are function objects, and `tol` and `max_iter` are strictly positive scalars. The input argument, `max_iter`, is also integer. The output argument, `X`, should be a scalar satisfying $|f(X) - g(X)| < tol$, that is, `X` is a point that (almost) satisfies $f(x) = g(x)$. To find `X`, you should use the bisection method with the error metric, $|f(m)| < tol$. The function `my_fixed_point` should "give up" after `max_iter` number of iterations and return $X = [\space]$ if this occurs.  

In [ ]:
import numpy as np
import math

def my_fixed_point(f, g, tol, max_iter):


  # Define our target function h(x)
  def h(x):
    return f(x) - g(x)


  # Choosing a and b points or interval [a, b]
  lower_bound = -100
  upper_bound = 100
  x = list((range(lower_bound, upper_bound)))

  for i in range(1, len(x)):
    if h(x[i])*h(x[i-1]) < 0:
      a = x[i-1]
      b = x[i]
      break

  # We implement a bisection method.
  def my_bisection(h, a, b, iter_count, tol):
  # get the midpoint
    iter_count = 0
    while iter_count < max_iter:
      m = (a + b)/2
      iter_count += 1
      if np.abs(h(m)) < tol:
    # stopping condition. Reports m as root
        return m
      elif np.sign(h(a)) == np.sign(h(m)):
    # case where m is an improvement on a
    # Make recursive call with a = m
        return my_bisection(h, m, b, iter_count, tol)
      elif np.sign(h(b)) == np.sign(h(m)):
    # case where m is an improvement on b.
    # Make recursive call with b = m
        return my_bisection(h, a, m, iter_count, tol)
    return my_bisection(h, a, b, iter_count, tol)

In [ ]:
f = lambda x: x**2
g = lambda x: x**3 - 5
tol = 1e-6
max_iter = 100
estimate = my_fixed_point(f, g, tol, max_iter)
print(f"Estimate = {estimate}")

Estimate = None


**3.** Write a function `my_bisection(f, a, b, tol)` that returns `[R, E]`, where `f` is a function object, `a` and `b` are scalars such that `a < b`, and `tol` is a strictly positive scalar value. The function should return an array, `R` where `R[i]` is the estimation of the root of $f$ defined by $(a+b)/2$ for the *i*-th iteration of the bisection method. Remember to include the initial estimate. The function should also return an array, `E`, where `E[i]` is the value of $|f(R[i])|$ for the *i*-th iteration of the bisection method. The function should terminate when `E(i) < tol`. Assume that $sign(f(a))\neq sign(f(b))$. Clarification: The input `a` and `b` constitute the first iteration of bisection; therefore, `R` and `E` should never be empty.   

In [ ]:
import numpy as np
import math

def my_bisection(f, a, b, tol):

  """
  Function to find a root in f
  that returns an R and E arrays
  with each iteration
  """
  # Raise an exception if a and b do not bound a root
  if np.sign(f(a)) == np.sign(f(b)):
    raise Exception(
        "The scalars a and b do not bound a root")

  # Initialize R and E as empty lists
  R = []
  E = []
  # We create a loop using "while True" functionality
  # until our desired condition
  while True:
    # Calculate root
    m = (a + b)/2
    # Append the results of each iteration
    R.append(m)
    E.append(np.abs(f(m)))
    # Making the comparison to narrow our search towards either a or b
    if np.abs(f(m)) < tol:
      break
    elif np.sign(f(m)) == np.sign(f(a)):
      a = m
    else:
      b = m
  return print(f"R = {R}"), print(f"E = {np.array(E, dtype = float).tolist()}")

In [ ]:
# Test 1
f = lambda x: x**2 - 2
R, E = my_bisection(f, 0, 2, 1e-1)

R = [1.0, 1.5, 1.25, 1.375, 1.4375]
E = [1.0, 0.25, 0.4375, 0.109375, 0.06640625]


In [ ]:
# Test 2
f = lambda x: np.sin(x) - np.cos(x)
R, E = my_bisection(f, 0, 2, 1e-2)


R = [1.0, 0.5, 0.75, 0.875, 0.8125, 0.78125]
E = [0.30116867893975674, 0.39815702328616975, 0.050050108850486774, 0.126546644072702, 0.038323093040207756, 0.005866372111545948]


**4.** Write a function `my_newton(f, df, x0, tol)` that returns `[R, E]`, where `f` is a function object, `df` is a function object giving the derivative of  `f`, `x0` is an initial estimation of the root, and `tol` is a strictly positive scalar. The function should return an array, `R`, where `R[i]` is the Newton_Raphson estimate of the root of `f`for the $i$th iteration. Remember to include the initial estimate. The function should also return an array, `E`, where `E[i]` is the value of $|f(R[i])|$ for the $i$th iteration of the Newton-Raphson method. The function should terminate when `E(i) < tol`. Assume
that the derivative of `f` will not hit zero during any iteration of the test cases given.




In [ ]:
def my_newton_raphson(f, df, x0, tol):
  """
  Function to find a root in f
  that returns an R and E arrays
  with each iteration. Newton-Raphson method
  """
  # Raise an exception if a and b do not bound a root
  if np.sign(df(x0)) == 0:
    raise Exception(
        "The method will not converge")

  # Initialize R and E containing the first iteration
  R = [x0]
  E = [np.abs(f(x0))]

  while True:
    # Calculate root using Nwthon_Raphson approximation
    x = x0 - f(x0)/df(x0)
    # Append results of each iteration
    R.append(x)
    E.append(np.abs(f(x)))
    # Condition to break the loop
    if np.abs(f(x)) > tol:
      x0 = x
    else:
      break

  return R, E #print(f"R = {R}\nE = {np.array(E, dtype = float).tolist()}")


In [ ]:
# Test case 1
f = lambda x: x**2 - 2
df = lambda x: 2*x
R, E = my_newton_raphson(f, df, 1, 1e-5)
print(f"R = {R}")
print(f"E = {E}")

R = [1, 1.5, 1.4166666666666667, 1.4142156862745099]
E = [np.int64(1), np.float64(0.25), np.float64(0.006944444444444642), np.float64(6.007304882871267e-06)]


In [ ]:
# Test case 2
f = lambda x: np.sin(x) - np.cos(x)
df = lambda x: np.cos(x) + np.sin(x)
R, E = my_newton_raphson(f, df, 1, 1e-5)
print(f"R = {R}")
print(f"E = {E}")

R = [1, np.float64(0.782041901539138), np.float64(0.7853981759997019)]
E = [np.float64(0.30116867893975674), np.float64(0.004746462127804163), np.float64(1.7822277875723103e-08)]
